In [ ]:
# Initialisation of libraries/directories
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as Img
import os
from PIL import Image
import cv2

from keras.models import *
from keras.layers import *

C:\Users\Maharshi Trivedi\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [ ]:
img=cv2.imread('dataset1/images_prepped_train/0001TP_006690.png')[:,:,::-1]
img=cv2.resize(img,dsize=(256,256),interpolation=cv2.INTER_NEAREST)
plt.imshow(img)

In [ ]:
img=Img.imread('dataset1/annotations_prepped_train/0001TP_006690.png') 
img=cv2.resize(img,dsize=(256,256),interpolation=cv2.INTER_NEAREST)
plt.imshow(img)

In [ ]:
image_test_name = os.listdir('dataset1/images_prepped_test')
image_train_name =os.listdir('dataset1/images_prepped_train')
mask_train_name =os.listdir('dataset1/annotations_prepped_train') 
mask_test_name = os.listdir('dataset1/annotations_prepped_test')

In [ ]:
image_test = []
image_train = []
mask_test = []
mask_train=[]
for j,l in zip(image_train_name, mask_train_name):
    img2 = cv2.imread(os.path.join('dataset1/images_prepped_train', j))[:,:,::-1]
    img2 = cv2.resize(img2,dsize=(256,256),interpolation=cv2.INTER_NEAREST) #Inter_nearest is important to have 12 classes
    img4 = Img.imread(os.path.join('dataset1/annotations_prepped_train',l))[:,::-1] #Img
    img4 = cv2.resize(img4,dsize=(256,256),interpolation=cv2.INTER_NEAREST)
    image_train.append(img2)
    mask_train.append(img4)

for i,k in zip(image_test_name,mask_test_name):
    img1 =  cv2.imread(os.path.join('dataset1/images_prepped_test', i))[:,:,::-1]
    img1 = cv2.resize(img1,dsize=(256,256),interpolation=cv2.INTER_NEAREST)
    img3 = Img.imread(os.path.join('dataset1/annotations_prepped_test', k))[:,::-1]
    a=img3
    img3 = cv2.resize(img3,dsize=(256,256),interpolation=cv2.INTER_NEAREST)
    image_test.append(img1)
    mask_test.append(img3)
    
image_test = np.array(image_test)
image_train = np.array(image_train)
mask_test = np.resize(np.array(mask_test),(367,256,256,1))
mask_train=np.resize(np.array(mask_train),(367,256,256,1))                    

In [ ]:
mask_train.shape

In [ ]:
plt.imshow(mask_test[0][:,:,0])

In [ ]:
np.unique(mask_train[225]).shape

In [ ]:
# No. of unique labels/annotations
labels=list(np.unique(mask_train[150]))

In [ ]:
a=np.squeeze(mask_train[0])
a[np.where(a==labels[1])]=1
a[np.where(a!=1)]=0
np.unique(a)

In [ ]:
plt.imshow(np.squeeze(mask_test[0]))

In [ ]:
# Generating mask from the label-annotations (Training Images)
mask_train_bw=np.full((mask_train.shape[0],mask_train.shape[1],mask_train.shape[2],len(labels)),0)
for image_no,m_train in enumerate(mask_train):
    for index,label in enumerate(labels):
        temp=np.full((mask_train.shape[1],mask_train.shape[2]),0)
        temp[np.squeeze(m_train)==label]=1
        mask_train_bw[image_no,:,:,index]=temp

In [ ]:
# Generating mask from the label-annotations (Testing Images)
mask_test_bw=np.full((mask_test.shape[0],mask_test.shape[1],mask_test.shape[2],len(labels)),0)
for image_no,m_test in enumerate(mask_test):
    for index,label in enumerate(labels):
        temp=np.full((mask_test.shape[1],mask_test.shape[2]),0)
        temp[np.squeeze(m_test)==label]=1
        mask_test_bw[image_no,:,:,index]=temp

In [ ]:
plt.imshow(mask_test_bw[9][:,:,10])

In [ ]:
mask_train_bw.shape

In [ ]:
mask_test_bw.shape

In [ ]:
#Keras Model

In [ ]:
import numpy as np
from keras.models import Sequential,Model
from keras.layers import Convolution2D, ZeroPadding2D, MaxPooling2D, Deconvolution2D, Cropping2D
from keras.layers import Input, Add, Dropout, Permute, add
from scipy.io import loadmat

from keras.preprocessing.image import ImageDataGenerator, load_img

In [ ]:
# Function to create to a series of CONV layers followed by Max pooling layer
def Convblock(channel_dimension, block_no, no_of_convs) :
    Layers = []
    for i in range(no_of_convs) :
        
        Conv_name = "conv"+str(block_no)+"_"+str(i+1)
        
        # A constant kernel size of 3*3 is used for all convolutions
        Layers.append(Convolution2D(channel_dimension,kernel_size = (3,3),padding = "same",activation = "relu",name = Conv_name))
    
    Max_pooling_name = "pool"+str(block_no)
    
    #Addding max pooling layer
    Layers.append(MaxPooling2D(pool_size=(2, 2), strides=(2, 2),name = Max_pooling_name))
    
    return Layers

In [ ]:
def FCN_8_helper(image_size):
    model = Sequential()
    model.add(Permute((1,2,3),input_shape = (image_size,image_size,3)))
    
    for l in Convblock(64,1,2) :
        model.add(l)
    
    for l in Convblock(128,2,2):
        model.add(l)
    
    for l in Convblock(256,3,3):
        model.add(l)
    
    for l in Convblock(512,4,3):
        model.add(l)
    
    for l in Convblock(512,5,3):
        model.add(l)
        
    model.add(Convolution2D(4096,kernel_size=(7,7),padding = "same",activation = "relu",name = "fc6"))
      
    #Replacing fully connnected layers of VGG Net using convolutions
    model.add(Convolution2D(4096,kernel_size=(1,1),padding = "same",activation = "relu",name = "fc7"))
    
    # Gives the classifications scores for each of the 21 classes including background
    model.add(Convolution2D(12,kernel_size=(1,1),padding="same",activation="relu",name = "score_fr"))
    
    Conv_size = model.layers[-1].output_shape[2] #16 if image size if 512
    print(Conv_size)
    
    model.add(Deconvolution2D(12,kernel_size=(4,4),strides = (2,2),padding = "valid",activation=None,name = "score2"))
    
    # O = ((I-K+2*P)/Stride)+1 
    # O = Output dimesnion after convolution
    # I = Input dimnesion
    # K = kernel Size
    # P = Padding
    
    # I = (O-1)*Stride + K 
    Deconv_size = model.layers[-1].output_shape[2] #34 if image size is 512*512
    
    #print(Deconv_size)
    # 2 if image size is 512*512
    Extra = (Deconv_size - 2*Conv_size)
    
    #print(Extra)
    
    #Cropping to get correct size
    model.add(Cropping2D(cropping=((0,Extra),(0,Extra))))
    
    return model
    
    

In [ ]:
def FCN_8(image_size):
    fcn_8 = FCN_8_helper(image_size)
    #Calculating conv size after the sequential block
    #32 if image size is 512*512
    Conv_size = fcn_8.layers[-1].output_shape[2] 
    
    #Conv to be applied on Pool4
    skip_con1 = Convolution2D(12,kernel_size=(1,1),padding = "same",activation=None, name = "score_pool4")
    
    #Addig skip connection which takes adds the output of Max pooling layer 4 to current layer
    Summed = add(inputs = [skip_con1(fcn_8.layers[14].output),fcn_8.layers[-1].output])
    
    #Upsampling output of first skip connection
    x = Deconvolution2D(12,kernel_size=(4,4),strides = (2,2),padding = "valid",activation=None,name = "score4")(Summed)
    x = Cropping2D(cropping=((0,2),(0,2)))(x)
    
    
    #Conv to be applied to pool3
    skip_con2 = Convolution2D(12,kernel_size=(1,1),padding = "same",activation=None, name = "score_pool3")
    
    #Adding skip connection which takes output og Max pooling layer 3 to current layer
    Summed = add(inputs = [skip_con2(fcn_8.layers[10].output),x])
    
    #Final Up convolution which restores the original image size
    Up = Deconvolution2D(12,kernel_size=(16,16),strides = (8,8),
                         padding = "valid",activation = None,name = "upsample")(Summed)
    
    #Cropping the extra part obtained due to transpose convolution
    final = Cropping2D(cropping = ((0,8),(0,8)))(Up)
    
    
    return Model(fcn_8.input, final)

In [ ]:
model = FCN_8(256)

In [ ]:
model.summary()

In [ ]:
from keras.optimizers import Adam, RMSprop
from keras.metrics import categorical_crossentropy
model.compile(optimizer=Adam(lr=.0001), loss= 'categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
history = model.fit(x=image_train,y=mask_train_bw, batch_size=50, epochs=2, verbose=2)

Epoch 1/2
